In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np

# Provided text
text = "Next character prediction is a fundamental task in the field of natural language processing (NLP) that involves predicting the next character in a sequence of text based on the characters that precede it. This task is essential for various applications, including text auto-completion, spell checking, and even in the development of sophisticated AI models capable of generating human-like text. At its core, next character prediction relies on statistical models or deep learning algorithms to analyze a given sequence of text and predict which character is most likely to follow. These predictions are based on patterns and relationships learned from large datasets of text during the training phase of the model. One of the most popular approaches to next character prediction involves the use of Recurrent Neural Networks (RNNs), and more specifically, a variant called Long Short-Term Memory (LSTM) networks. RNNs are particularly well-suited for sequential data like text, as they can maintain information in 'memory' about previous characters to inform the prediction of the next character. LSTM networks enhance this capability by being able to remember long-term dependencies, making them even more effective for next character prediction tasks. Training a model for next character prediction involves feeding it large amounts of text data, allowing it to learn the probability of each character's appearance following a sequence of characters. During this training process, the model adjusts its parameters to minimize the difference between its predictions and the actual outcomes, thus improving its predictive accuracy over time. Once trained, the model can be used to predict the next character in a given piece of text by considering the sequence of characters that precede it. This can enhance user experience in text editing software, improve efficiency in coding environments with auto-completion features, and enable more natural interactions with AI-based chatbots and virtual assistants. In summary, next character prediction plays a crucial role in enhancing the capabilities of various NLP applications, making text-based interactions more efficient, accurate, and human-like. Through the use of advanced machine learning models like RNNs and LSTMs, next character prediction continues to evolve, opening new possibilities for the future of text-based technology."

# Creating character index mappings
chars = sorted(set(text))
char_to_index = {char: index for index, char in enumerate(chars)}
index_to_char = {index: char for index, char in enumerate(chars)}

# Preparing the dataset
max_length = 10
step = 1
sequences = []
next_chars = []

for i in range(0, len(text) - max_length, step):
    sequences.append([char_to_index[char] for char in text[i: i + max_length]])
    next_chars.append(char_to_index[text[i + max_length]])

# Vectorization
X = np.zeros((len(sequences), max_length, len(chars)), dtype=np.float32)
y = np.zeros((len(sequences), len(chars)), dtype=np.float32)

for i, sequence in enumerate(sequences):
    for t, char_index in enumerate(sequence):
        X[i, t, char_index] = 1
    y[i, next_chars[i]] = 1

# Define LSTM model
model = Sequential([
    LSTM(128, input_shape=(max_length, len(chars))),
    Dense(len(chars), activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Training the model
model.fit(X, y, batch_size=128, epochs=10)

# Prediction function
def predict_next_char(model, input_text, char_to_index, index_to_char):
    x_pred = np.zeros((1, max_length, len(chars)))
    input_text = input_text[-max_length:]  # Ensure input is not longer than max_length
    for t, char in enumerate(input_text):
        if char in char_to_index:  # Check if char is known
            x_pred[0, t, char_to_index[char]] = 1.
    preds = model.predict(x_pred, verbose=0)[0]
    next_index = np.argmax(preds)
    next_char = index_to_char[next_index]
    return next_char

# Example prediction
test_start = "Next character prediction is a fundamental task"
print(f"Predicted next character for '{test_start}' is: '{predict_next_char(model, test_start, char_to_index, index_to_char)}'")


Epoch 1/10
19/19 [==============================] - 1s 13ms/step - loss: 3.5904
Epoch 2/10
19/19 [==============================] - 0s 4ms/step - loss: 3.0635
Epoch 3/10
19/19 [==============================] - 0s 4ms/step - loss: 2.9987
Epoch 4/10
19/19 [==============================] - 0s 4ms/step - loss: 2.9829
Epoch 5/10
19/19 [==============================] - 0s 4ms/step - loss: 2.9601
Epoch 6/10
19/19 [==============================] - 0s 4ms/step - loss: 2.9356
Epoch 7/10
19/19 [==============================] - 0s 4ms/step - loss: 2.9033
Epoch 8/10
19/19 [==============================] - 0s 4ms/step - loss: 2.8600
Epoch 9/10
19/19 [==============================] - 0s 4ms/step - loss: 2.8160
Epoch 10/10
19/19 [==============================] - 0s 5ms/step - loss: 2.7781
Predicted next character for 'Next character prediction is a fundamental task' is: ' '
